In [1]:
%pip install beautifulsoup4
%pip install playwright
# installs playwright for different web browsers using the command line '!'
!playwright install

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os

SEASONS = list(range(2016,2023))

DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [2]:
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time
# Make sure to install playwright browsers by running playwright install on the command line or !playwright install from Jupyter

In [4]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [5]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [6]:
for season in SEASONS:
    await scrape_season(season)

Task exception was never retrieved
future: <Task finished name='Task-5' coro=<Connection.run() done, defined at C:\Users\henry\anaconda3\lib\site-packages\playwright\_impl\_connection.py:303> exception=NotImplementedError()>
Traceback (most recent call last):
  File "C:\Users\henry\anaconda3\lib\site-packages\playwright\_impl\_connection.py", line 310, in run
    await self._transport.connect()
  File "C:\Users\henry\anaconda3\lib\site-packages\playwright\_impl\_transport.py", line 133, in connect
    raise exc
  File "C:\Users\henry\anaconda3\lib\site-packages\playwright\_impl\_transport.py", line 120, in connect
    self._proc = await asyncio.create_subprocess_exec(
  File "C:\Users\henry\anaconda3\lib\asyncio\subprocess.py", line 236, in create_subprocess_exec
    transport, protocol = await loop.subprocess_exec(
  File "C:\Users\henry\anaconda3\lib\asyncio\base_events.py", line 1676, in subprocess_exec
    transport = await self._make_subprocess_transport(
  File "C:\Users\henry\an

NotImplementedError: 

The following script "scraper.py" has the main functions to go through the website's html pages and pull and download the html pages to our storage. Further functions can be used to process these html pages and pull important data from the page's tables into a pandas dataframe.

Since playwright opens a browser on the side (which takes time), playwright runs asyncronously. This means that functions are run on another thread and will return when the function is complete. Jupyter notebook on Windows is unable to process asyncronous playwright's event loop as JN is running it's own event loop which clashes.

Running a python script however counteracts this error.

In [4]:
!python scraper.py

C:\Users\henXc\NBA-Predictor-main\NBA-Predictor-main\scraper.py:58: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 58 of the file C:\Users\henXc\NBA-Predictor-main\NBA-Predictor-main\scraper.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html)


In [ ]:
#loop through every .html file in the scores directory
#add the file to the relative season folder based on date of the match
#if month lies between 9-12 month n-1 year or 1-7 month n year, then place file in n-1 - n folder